## Hyperparameter Tuning for CNN

### Imports

In [ ]:
import csv
import math

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Load image paths, prices, and train/test split

In [ ]:
# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/bikes_im/"
with open("../datasets/bikes_filtered.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[2]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(int(msrp))

train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

### Initialize Image Generator

Due to the size of our dataset (>20,000 images), we cannot read all images into memory. Thus, we write our own image generator, which is a Python generator that reads images a minibatch at a time, preprocessing them and returning the input data and price labels as input to the neural network.

In [ ]:
def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 1))
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 1))
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)                
                Y[i] = prices[index]
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

### Hyperparameters

We tune hyperparameters using grid search and random search, modifying one hyperparameter at a time while keeping the others constant.

In [ ]:
# Hyperparameters

num_settings = 1

hp_dropout = [0.2] * num_settings

#RMSprop
hp_lr = [0.001] * num_settings
hp_rho = [0.9] * num_settings
hp_epsilon = [1e-07] * num_settings
hp_decay = [0.0] * num_settings

# Number of hidden units
hp_hidden = [256] * num_settings

# Minibatch size
hp_mbsize = [64] * num_settings

num_epochs = 20

In [ ]:
# store the results of each setting
train_losses = np.zeros(num_settings)
dev_losses = np.zeros(num_settings)

for setting in range(num_settings):
    # build the VGG16 network
    input_tensor = Input(shape=(224,224,3))
    model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)
    
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=(model.output_shape[1:])))


    # Output layer
    # We do random weight intialization
    top_model.add(Dropout(hp_dropout[setting]))
    top_model.add(Dense(hp_hidden[setting], activation='relu', kernel_initializer='glorot_uniform'))
    top_model.add(Dense(1, activation='linear', name='output', kernel_initializer='glorot_uniform'))
    
    # add the model on top of the convolutional base
    new_model = Model(inputs= model.input, outputs = top_model(model.output))

    # set the first 19 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in new_model.layers[:19]:
        layer.trainable = False

    # RMSprop
    new_model.compile(loss='mean_squared_error',
                      optimizer=optimizers.RMSprop(
                              lr=hp_lr[setting], 
                              rho=hp_rho[setting], 
                              epsilon=hp_epsilon[setting], 
                              decay=hp_decay[setting]))
    
    epochs = 10
    minibatch_size = hp_mbsize[setting]

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)

    # fine-tune the model
    history = new_model.fit_generator(
        image_generator(train_indices, minibatch_size),
        steps_per_epoch=train_steps,
        epochs=num_epochs,
        validation_data=image_generator(test_indices, minibatch_size),
        nb_val_samples=test_steps)
    
    # store the training and dev losses for the last epoch (current model)
    train_losses[setting] = history.history['loss'][-1]
    dev_losses[setting] = history.history['val_loss'][-1]
    

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()
    
    print("==========")

### TODO(stevenzc, rry): Write a bit of code to run MSE, MAE, and R2 on the test set.

In [ ]:
# Take a data point and run it through the neural network
# Return the predicted value and calculate the MSE
def evaluate(model, index):
    msrp = prices[index]
    path = image_paths[index]
    img = image.load_img(path, target_size=(224, 224))
    data = np.expand_dims(image.img_to_array(img), axis=0)
    
    # TODO(stevenzc, rry): the input should be preprocessed before feeding it in
    
    prediction = model.predict(data, msrp)
    
    #print("Bike index: " + str(index))
    #print("Actual price: " + str(msrp))
    #print("Predicted price: " + str(prediction))
    mse = (prediction-msrp)**2
    return mse
